In [2]:
import os,sys,time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import parallel

## Exploración de configuraciones de estimadores

- usando Suppor Vector Machines y Decision Trees
- usa el `metadata` para ver la distribución de tareas y mide cuánto tardó cada tarea en ejecutarse
- haz el ejercicio con `DirectView` y con `LoadBalancedView`

In [213]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

estimators = []
for gamma in [0.01, 0.05]: #, 0.1, 0.3, 0.6, 0.9, 1., 1.5, 2.]:
    for C in [0.01, 0.1]: #, 0.5, 1., 10.]:
        estimators.append(SVC(gamma=gamma, C=C))
        
for max_depth in [1,2,3,4,5]:
    estimators.append(DecisionTreeClassifier(max_depth=max_depth))


In [181]:
print len(estimators)

9


In [182]:
rc = parallel.Client()
print "available engines", rc.ids

available engines [1, 2]


## TAREA 1: Paraleliza con `DirectView` y `%px`

Usamos un DirectView para:
- enviar `d` y `c` a todos los engines
- hacer un scatter de la lista de estimadores

Usamos `%px` para:
- definir la función `fit_estimator` en los engines
- ejecutar la función `fit_estimator` con la porción de la lista de estimadores que le tocó a cada engine

In [183]:
dv = rc.direct_view()

# --- TU CODIGO AQUI: ENVIA d y c a todos los engines

# --- TU CODIGO AQUI: DISTRIBUYE estimators ENTRE LOS ENGINES


In [114]:
%%px
from sklearn import cross_validation
import numpy as np
def fit_estimator(estimator, d, c, test_size):
    d_train, d_test, c_train, c_test = cross_validation.train_test_split(d, 
                                                                         c, 
                                                                         test_size=test_size, 
                                                                         random_state=np.random.randint(1,100))
    estimator.fit(d_train, c_train)
    return np.array([estimator.score(d_train, c_train), estimator.score(d_test, c_test)])

In [ ]:
## -- TU CODIGO AQUI
## -- ejecuta la función fit_estimator en cada engine con %px 
## -- usando la porcion de los estimators que le toco a cada uno

## TAREA 2: Paraleliza con `DirectView` y `map`

- además inspeccionamos el metadata de la ejecución

In [199]:
dv.clear()

In [201]:
## -- TU CODIGO AQUI
## -- ejecuta la función fit_estimator en cada engine con dv.map
## -- usando la porcion de los estimators que le toco a cada uno
r = ...

In [202]:
rs = r.get()


fíjate como ahora inspeccionamos los metadatos de la ejecución

In [ ]:
for i in range(len(r.metadata)):
    tsub = r.metadata[i]["started"]
    tcom = r.metadata[i]["completed"]
    print "task", i, (tcom-tsub).total_seconds(), "secs"


## TAREA 3: `LoadBalancedView` con `map`

In [209]:
lb = rc.load_balanced_view()

In [207]:
## -- TU CODIGO AQUI
## -- ejecuta la función fit_estimator en cada engine con lb.map
## -- usando la porcion de los estimators que le toco a cada uno
r = ...


In [ ]:
rs = r.get()
print rs

In [ ]:
for i in range(len(r.metadata)):
    tsub = r.metadata[i]["started"]
    tcom = r.metadata[i]["completed"]
    print "task", i, (tcom-tsub).total_seconds(), "secs"
